# Jupyter Widget Animation Backend Examples

This notebook demonstrates the widget backend for interactive animation in Jupyter notebooks. The widget backend provides:

- **Interactive controls**: Play button + slider for frame scrubbing
- **Smart caching**: Pre-renders first 500 frames for responsive scrubbing
- **On-demand rendering**: Frames beyond cache rendered when accessed
- **Smooth scrubbing**: `continuous_update=True` for real-time frame updates
- **JavaScript linking**: High-performance widget synchronization

## Requirements

```bash
pip install ipywidgets
# or: uv add ipywidgets
```

In [1]:
import numpy as np

from neurospatial import Environment
from neurospatial.animation.backends.widget_backend import render_widget

## Example 1: Basic Widget Animation

Create a simple animated field with interactive controls.

In [2]:
# Create a 2D environment
positions = np.random.randn(100, 2) * 50
env = Environment.from_samples(positions, bin_size=10.0)
print(f"Environment: {env.n_bins} bins, 2D")

# Create animated fields (simulating a moving activity bump)
n_frames = 50
fields = []
for i in range(n_frames):
    center = np.array([np.sin(i / 5) * 40, np.cos(i / 5) * 40])
    field = np.exp(-np.sum((env.bin_centers - center) ** 2, axis=1) / 200)
    fields.append(field)

print(f"Created {n_frames} frames")

Environment: 85 bins, 2D
Created 50 frames


In [ ]:
# Launch widget with default settings
widget = render_widget(env, fields, fps=10)

# Note: Widget displays automatically above this cell
# Controls:
#   - Play button (▶): Automatic playback at 10 FPS
#   - Slider: Manual frame scrubbing (drag to any frame)
#   - Frame counter: Shows "Frame X" label

Pre-rendering 50 frames for widget...


interactive(children=(IntSlider(value=0, description='Frame:', max=49), Output()), _dom_classes=('widget-inter…

## Example 2: Custom Frame Labels

Add meaningful labels to each frame (e.g., trial numbers, time stamps).

In [ ]:
# Create fields for multiple "trials"
n_trials = 5
frames_per_trial = 10
n_frames_total = n_trials * frames_per_trial

fields_trials = []
labels = []

for trial in range(n_trials):
    for frame in range(frames_per_trial):
        # Different center for each trial
        angle = (trial / n_trials) * 2 * np.pi
        center = np.array([np.cos(angle) * 30, np.sin(angle) * 30])

        # Activity moves within trial
        offset = np.array([np.sin(frame / 3) * 10, np.cos(frame / 3) * 10])
        field = np.exp(
            -np.sum((env.bin_centers - (center + offset)) ** 2, axis=1) / 150
        )
        fields_trials.append(field)

        # Custom label
        time_sec = frame * 0.1
        labels.append(f"Trial {trial + 1} - {time_sec:.1f}s")

print(f"Created {len(fields_trials)} frames with custom labels")

Created 50 frames with custom labels


In [5]:
# Launch widget with custom labels
widget = render_widget(env, fields_trials, frame_labels=labels, fps=20, cmap="hot")

# Note: Frame labels display above each frame image
# Try scrubbing through to see different trials!

Pre-rendering 50 frames for widget...


interactive(children=(IntSlider(value=0, description='Frame:', max=49), Output()), _dom_classes=('widget-inter…

## Example 3: Custom Color Scale

Control the colormap normalization across all frames.

In [6]:
# Create fields with varying intensity
fields_varying = []
for i in range(30):
    scale = 0.5 + 0.5 * np.sin(i / 3)  # Intensity varies from 0.0 to 1.0
    center = np.array([0, 0])
    field = np.exp(-np.sum((env.bin_centers - center) ** 2, axis=1) / 200) * scale
    fields_varying.append(field)

print(f"Created {len(fields_varying)} frames with varying intensity")

Created 30 frames with varying intensity


In [7]:
# Launch with custom color scale (fixed normalization)
widget = render_widget(
    env,
    fields_varying,
    fps=15,
    cmap="plasma",
    vmin=0.0,
    vmax=1.0,  # Fix max to 1.0 (consistent across frames)
)

# Note: Colors are normalized consistently across all frames
# Compare this to auto-normalization (default) where each frame
# is normalized independently

Pre-rendering 30 frames for widget...


interactive(children=(IntSlider(value=0, description='Frame:', max=29), Output()), _dom_classes=('widget-inter…

## Example 4: High-Resolution Rendering

Increase DPI for higher quality frames (larger file size).

In [8]:
# Create a smaller set of frames for high-res demo
n_frames_hires = 20
fields_hires = []
for i in range(n_frames_hires):
    center = np.array([np.sin(i / 3) * 30, np.cos(i / 3) * 30])
    field = np.exp(-np.sum((env.bin_centers - center) ** 2, axis=1) / 150)
    fields_hires.append(field)

In [9]:
# Launch with high DPI (higher resolution, larger memory footprint)
widget = render_widget(
    env,
    fields_hires,
    fps=10,
    dpi=150,  # Default is 100
    cmap="viridis",
)

# Note: Higher DPI = sharper image but larger pre-rendered cache
# Default DPI (100) is good for most use cases

Pre-rendering 20 frames for widget...


interactive(children=(IntSlider(value=0, description='Frame:', max=19), Output()), _dom_classes=('widget-inter…

## Example 5: Large Dataset (Cache Demo)

Demonstrate the widget's caching strategy with a larger dataset.

In [10]:
# Create a larger dataset (600 frames - exceeds cache size of 500)
n_frames_large = 600
print(f"Creating {n_frames_large} frames...")

fields_large = []
for i in range(n_frames_large):
    t = i / 50
    center = np.array([np.sin(t) * 40, np.cos(t) * 40])
    field = np.exp(-np.sum((env.bin_centers - center) ** 2, axis=1) / 200)
    fields_large.append(field)

print(f"Created {len(fields_large)} frames")
print("Note: Widget will pre-render first 500 frames, then render on-demand")

Creating 600 frames...
Created 600 frames
Note: Widget will pre-render first 500 frames, then render on-demand


In [11]:
# Launch widget - watch pre-rendering progress
widget = render_widget(env, fields_large, fps=30, cmap="hot")

# Try this:
# 1. Scrub through first 500 frames - should be instant (pre-cached)
# 2. Jump to frame 550+ - slight delay as frame renders on-demand
# 3. Scrub back to earlier frames - instant again (still cached)

Pre-rendering 500 frames for widget...


interactive(children=(IntSlider(value=0, description='Frame:', max=599), Output()), _dom_classes=('widget-inte…

## Performance Notes

### Widget Backend Features

✓ **Pre-rendering**: First 500 frames cached during initialization  
✓ **On-demand rendering**: Frames beyond cache rendered when accessed  
✓ **Smooth scrubbing**: `continuous_update=True` for real-time frame updates  
✓ **JavaScript linking**: High-performance synchronization between play button and slider  
✓ **Memory efficient**: ~50-100 MB for 500 pre-rendered frames (depends on DPI)  

### When to Use Widget Backend

**Use widget backend when:**
- Working in Jupyter notebooks
- Need interactive controls (play/pause, scrubbing)
- Dataset size: 10-1000 frames (sweet spot)
- Want to share notebooks with embedded animations

**Use other backends when:**
- **Napari**: >1000 frames, need GPU acceleration, want 3D viewer
- **Video**: Need MP4 export for presentations/papers
- **HTML**: Need standalone file to share (no Jupyter required)

### Memory Considerations

- **500 frames @ DPI 100**: ~50-100 MB (depends on environment size)
- **500 frames @ DPI 150**: ~100-150 MB
- Frames beyond cache: rendered on-demand (no additional memory)

### Tips for Large Datasets

If working with >1000 frames:
1. Consider using Napari backend instead (GPU-accelerated, better for large datasets)
2. Or subsample frames before rendering:
   ```python
   from neurospatial.animation import subsample_frames
   fields_subsampled = subsample_frames(fields, source_fps=250, target_fps=30)
   ```
3. Reduce DPI (e.g., `dpi=80`) to decrease cache memory

### Jupyter Widget Controls

The widget provides:
- **▶ Play button**: Automatic playback at specified FPS
- **Slider**: Manual frame scrubbing (drag to jump to any frame)
- **Frame counter**: Current frame index (0 to n_frames-1)
- **Frame label**: Custom label if provided, or "Frame X" by default

### Backend Comparison

| Feature | Napari | Widget | Video | HTML |
|---------|--------|--------|-------|------|
| Interactive | ✓ | ✓ | ✗ | ✓ |
| GPU accelerated | ✓ | ✗ | ✗ | ✗ |
| Jupyter native | ✗ | ✓ | ✗ | Partial |
| Max frames | 100K+ | 1000 | ∞ | 500 |
| Shareable | ✗ | Notebook | ✓ | ✓ |
| Setup | Qt required | ipywidgets | ffmpeg | None |
